## So you want to do research on Twitter

This tutorial covers the basics of using the _twitter_ library in Python to perform searches using the REST API.  

__NB:__ There are a lot of different ways of accessing and processing Twitter data. I'm going to go through my workflow, which is a kludging together of methods assembled over the last four years.  There are probably better approaches to at least some parts of what I'm going through here, and if you know of one or find one, let me know!

## What's Twitter like?

Not to sound like a tech evangelist, but one neat thing about Twitter is that everyone uses and experiences it differently.  (Let's have an aside about everyone's familiarity with Twitter and what they use it for.)

Now, here's the interesting thing to me about using Twiter for research: Twitter is much more mundane than you think. It's a little paradoxical; most tweets on Twitter are nothing like most tweets that we see, because the tweets that pull big numbers (views, favs, RTs) are from bigger accounts and/or disseminated through the network broadly. But most tweets, like most of us, die in obscurity.

Here's an example of this.  Let's search for something really pragmatically empty, so we're going to get a scattering of topics. Specifically, let's look at tweets containing the word *the*:

https://twitter.com/search?l=&q=the&src=typd

The usual, common suspects show up in the default web-based search. Trump, 5 Seconds of Summer, news organizations, etc.  But that's because the Twitter algorithm defaults to show you a mix of recent and "top" tweets.  If you click on the little "Latest" tab, it gives you all the recent tweets for *the*, and it looks a lot different. 

https://twitter.com/search?f=tweets&vertical=default&q=the&src=typd

Here are the normies, the folks like you or me, or at least me, who chat with a few friends or try in vain to get noticed. Now, depending on your goals in your research, you might want the popular accounts and the viral tweets.  But if you're interested in how people talk, or how they interact, it's these normies who are your best friends.

PS: if you want to test out your searches before APIing them, here's the advanced search link: https://twitter.com/search-advanced


## Initialization basics

We'll start by setting up a Twitter search object, and doing a basic search.

In [9]:
import sys, os
from pprint import pprint                           #Important for reading through JSONs
from time import localtime,strftime,sleep,time      #Important for dealing with Twitter rate limits
import twitter

### Authorizing your account

The first thing you need is to perform an OAuth dance to tell Twitter who you are, and that you're cool. This is a slightly complicated step, involving identification both of a user conducting the searches and a client through which they're conducting them.

Hopefully you have already set up both your Twitter account and an app for accessing this data. If not, do it now; it's a pretty trivial process once you have a Twitter account, and can be set up in a few minutes here: https://apps.twitter.com/app/new

[Optional aside with link to set up app.]

Now that you have an app set up, we're going to input its access keys. Go to https://apps.twitter.com, click on your app, and then click on the "Keys and Access Tokens" tab.  Copy them in when prompted.

In [2]:
cons_oauth_file = 'c.xxx'
if os.path.exists(cons_oauth_file):
    constoken, conssecret = twitter.read_token_file(cons_oauth_file)
else:
    constoken = raw_input("What is your app's 'Consumer Key'?").strip()
    conssecret = raw_input("What is your app's 'Consumer Secret'?").strip()
    wf = open(cons_oauth_file,'w'); wf.write(constoken+'\n'+conssecret); wf.close()

Now we'll authorize this app through your Twitter account. This code block will open a new tab, prompt to activate the app, and provide you with a PIN to enter back here. This OAuth registration is handled by the *twitter.oauth_dance* function.

In [3]:
app_oauth_file = 'a.xxx'
if not os.path.exists(app_oauth_file):									#if user not authorized already
	twitter.oauth_dance("your app",constoken,conssecret,app_oauth_file)		#perform OAuth Dance
apptoken, appsecret = twitter.read_token_file(app_oauth_file)					#import user credentials

### Initializing a search object

Here's the core of the code, really -- we're setting up an object that uses your credentials to access the Twitter REST API and return structured Twitter data.  Thanks to the _twitter_ library, it's a single command, and then we can work with the resulting object, _tsearch_.

In [4]:
tsearch = twitter.Twitter(auth=twitter.OAuth(apptoken,appsecret,constoken,conssecret))	#create search command

## Basics of the search object & API

Now that _tsearch_ is initialized, let's get to searching! Except, actually, let's hold up a second and talk about how to use the API.

Twitter provides fairly extensive (but often confusing) documentation for its REST API, available here: https://dev.twitter.com/rest/reference  Basically, the REST API handles searches of recent tweets (the last week, generally), as well as functions related to the social network structure, trending topics, and user information.

### An aside: what won't we be covering?

There is another Twitter API, the streaming API, which maintains a connection to Twitter and collects tweets as they occur.  I don't have enough experience with the streaming API to teach it, and many of its potential applications can be handled through the REST API.  I also won't be dealing with the POST features of the REST API, because we're focused on collecting existing data rather than generating our own.

### Rate limits

Perhaps the most important thing to know about for the REST API is that it is rate-limited. For each app/account combo, you can only perform so many searches per time period. I believe Twitter resets its counter every 15 minutes for all API calls; it definitely does for all the common ones.  If you try to search once you've hit the rate limit, the search will return an error. You will want to make sure your code doesn't persist much after it hits a rate limit, or else Twitter may get mad at your account (this is why I suggest setting up an account specifically for these searches, just in case).

There is also a return limit, both as a limit to the number of results any single search will give you, and a limit on how far back Twitter will search for you.  In general, you can't ask for more than 100 hits per individual search, though you can search reatedly to get more and more 100-hit blocks.  But after a while, Twitter says enough of that and stops giving you anything new.  We'll get to illustrations in a little bit, but for now, here are some commonly encountered limits, and the full list is available [here](https://dev.twitter.com/rest/public/rate-limits).

| Function call | limit per 15 minutes | max returns per search | max history | 
|---|---|---|
| GET search/tweets | 180 | 100 | 3000 / 7 days |
| GET statuses/user_timeline | 900 | ? | 3200 / infinite past |
| GET users/lookup | 900 | 100 | NA |

### Checking rate limits

Since we don't want to anger Twitter, let's talk about checking rate limits.  Your _tsearch_ object can check rate limits, and it's a nice way of getting used to the nested list format of the search results.

By the way, checking rate limits **is itself rate limited** (180 / 15 minutes)! 

In [6]:
r = tsearch.application.rate_limit_status()  #Obtain rate limit status for all searches
pprint(r)

{u'rate_limit_context': {u'access_token': u'875840141905936384-mHcprEkKJs6WUpQ0ImHTJTctLWzRrZP'},
 u'resources': {u'account': {u'/account/login_verification_enrollment': {u'limit': 15,
                                                                         u'remaining': 15,
                                                                         u'reset': 1498163071},
                             u'/account/settings': {u'limit': 15,
                                                    u'remaining': 15,
                                                    u'reset': 1498163071},
                             u'/account/update_profile': {u'limit': 15,
                                                          u'remaining': 15,
                                                          u'reset': 1498163071},
                             u'/account/verify_credentials': {u'limit': 75,
                                                              u'remaining': 75,
                                

*application.rate_limit_status()* returns a set of nested dictionaries; one for each type of API request, sorted by request type. For each request, it lists the rate limit (*'limit'*), the number of API calls you have left before you hit that limit (*'remaining'*), and the Unix time at which your rate limit resets (*'reset'*).

Each API call type is tracked completely separately, and the rate-limit window is **15 minutes from your first call *specific to that request*.**  For instance, let's try doing a standard term-based search (GET search/tweets) and then check the rate limits again.

In [7]:
res = tsearch.search.tweets(q='test',              #A test query
                            count=10,              #just want 10 hits back
                            result_type="recent")  #include all recent tweets, not only popular ones

r = tsearch.application.rate_limit_status()  #Obtain rate limit status for all searches
pprint(r['resources']['application'])        #rate limit query's rate limit
pprint(r['resources']['search'])             #search query's rate limit

{u'/application/rate_limit_status': {u'limit': 180,
                                     u'remaining': 178,
                                     u'reset': 1498163071}}
{u'/search/tweets': {u'limit': 180, u'remaining': 179, u'reset': 1498163597}}


We've now done two rate limit queries, so the remaining available in this window is 178. But we've only done one term-based query, so there are still 179 left on that.  But importantly, the two have different reset times, because the rate-limit reset was triggered by the first search from the previous code block, while the term search was triggered more recently.

By the way, the reset time is always given in Unix time, UTC (i.e., number of milliseconds since 1/1/1970 UTC). Converting to local human-readable time isn't too bad, using *localtime* and *strftime* from the Python *time* library:

In [18]:
print('Rate limit reset:')
print(localtime(r['resources']['application']['/application/rate_limit_status']['reset']))
print(strftime('%I:%M:%S',localtime(r['resources']['application']['/application/rate_limit_status']['reset'])))
print('\n')
print('Term search reset:')
print(strftime('%I:%M:%S',localtime(r['resources']['search']['/search/tweets']['reset'])))

Rate limit reset:
time.struct_time(tm_year=2017, tm_mon=6, tm_mday=22, tm_hour=13, tm_min=24, tm_sec=31, tm_wday=3, tm_yday=173, tm_isdst=1)
01:24:31


Term search reset:
01:33:17


### Dealing with the rate limit

In this tutorial, we won't push up against the rate limit much, but for many tasks, you're going to have to deal intelligently with it. Let me share a few quick tricks, and then we'll move on to real searches.

**Trick 1: *time.sleep()* **

The most basic solution is to take advantage of the *sleep* function, which tells Python to hold on for some specified length of time. When your 'remianing' value drops to (or near) zero, record the 'reset' time, and subtract the current time from it (*time.time()*).  I generally add a little buffer time, too, in case my system clock is off from Twitter's.

One note: if you haven't triggered a rate limit window by doing a search, the reset value will be 15 minutes from now, so build in a check to make sure that you're not sitting around infinitely.  Any suggestions for how to best handle that?

In [22]:
print 'Reset time:  ',r['resources']['application']['/application/rate_limit_status']['reset']
print 'Current time:',time()
print 'Wait time:   ',r['resources']['application']['/application/rate_limit_status']['reset'] - time() + 30

Reset time:   1498163071
Current time: 1498165018.64
Wait time:    -1917.64221597


**Trick 2: interleave requests**

Because the rate limits are triggered separately for each search type, if you're doing multiple search types (say, term and user searches), you can switch back and forth, working with one until the limit's hit, then going to the other.

In [27]:
temp = tsearch.followers.list()
r = tsearch.application.rate_limit_status()
pprint(r['resources']['followers'])

{u'/followers/ids': {u'limit': 15, u'remaining': 15, u'reset': 1498165552},
 u'/followers/list': {u'limit': 15, u'remaining': 12, u'reset': 1498165478}}


**Trick 3: process in the moment**

If you're doing any substantial tweet processing within Python, you can take advantage of the rate limit to do time-consuming processing while you wait. Since the rate limit window is triggered by the first search, you want to make sure to put the first search as early in your code as possible, but subsequent searches can come either before or after processing wihtout consequnces.

**Pitfall 1: track limits internally, confirm occasionally**

One small thing: since the rate limit query is itself rate-limited, you want to ration your calls to it, especially if you're performing more than 180 searches by interleaving request types. I keep an internal count of the number of times I called each search, and then every 3 or so searches, check back with *application.rate_limit_status()* to make sure twitter and I are in agreement.

### What happens if we pass the rate limit?

All right, let's give in to our darker motivations and wantonly exceed the rate limit to find out the consequences.



In [33]:
r = tsearch.application.rate_limit_status()
for i in range(0,r['resources']['followers']['/followers/list']['remaining']):
    temp = tsearch.followers.list()

In [39]:
try:
    temp = tsearch.followers.list()
except:
    print "error"

error
{u'next_cursor': 0,
 u'next_cursor_str': u'0',
 u'previous_cursor': 0,
 u'previous_cursor_str': u'0',
 u'users': []}


Ah, it just kinda hangs there! So maybe that sets up:

**Trick 5: always *try* **

Unfortunately, the *twitter* library doesn't handle errors very well, in part, I think, because Twitter changes how the API handles errors from time to time.  So, if you're going to be doing a large number of searches and/or automating your system, be sure to build some safety valves in the form of *try-catch-except* statements around all the non-manual searches.

Note that sometimes the system itself craps out, so not all errors are due to rate limit violations. For important searches that I'm not observing, after a failed search, I have it wait a few seconds and try again (and maybe even another wait-try cycle), before doing a 15-minute wait cycle to make sure all the rate limits have refreshed.  You can probably work out a better system than that.

## Actually searching

Okay, enough of all the boring limitations. Let's take this puppy out on the road in Section 2!

In [2]:
#Global variables
loclist = ["30.8,-98.6","39.8,-95.6","32.8,-117.6","37.8,-122.6"]
radius = "2500km"
multiloc = True
tweetspersearch = 75
maxpages = 20
maxid = float("+inf")
importcsv = ''
overwrite = 'w'
outfile = ''
keeptweets = True
startat = 0
header = True
throttle = True
checklimits = 3 		#To avoid overquerying the rate_limit function, only check every N iterations
importmultiloc = False
newmultiloc = False
onlyincltweets = True
wff = False
trackfails = False
MAXERRORS = 3
scheduled = False
raw = True
tweetcount = 0

### Setting up your account to use SeeTweet

First things first, you need a Twitter account through which to access the REST API. Just in case you run afoul of any Twitter regulations---and it's not inconceivable as you start out---I recommend using an account you could survive being Twitter jailed.

In [3]:
tsearch = authorize()

### Choosing a search term

Okay, we're going to look for some term here. You can look for individual words or a phrase.

In [4]:
term1 = '"hot outside"'
term1 = re.sub(' ','+',term1)
term = re.sub('\"','%22',term1)

print "Searching for", term

Searching for %22hot+outside%22


### Main code

This will be revised to simply & improve transparency. Ideally will be reduced to simple API calls with reduced opaque processing.  Some additional processing I'm currently doing will also be removed.

Good feature to add here is URL for each discovered tweet in case you want to look at what's going on as it comes out.

In [5]:
#Basic startup processing
if (importcsv and multiloc):
	importmultiloc = True
elif multiloc:
	newmultiloc = True


if importcsv:
	rf = open(importcsv,'r')
	firstline=True
	tidnum = 0
	tids = []
	centers = []
	incls = []
	for line in rf:
		if line[0] == '#':
			continue
		splitline = line.strip().split(',')
		if firstline:
			tidnum = splitline.index('tid')
			if importmultiloc:
				centernum = splitline.index('center')
				inclnum = splitline.index('incl')
			firstline=False
		else:
			if onlyincltweets:							#If we're excluding excluded tweets from baseline calc, skip to next line if incl=0
				if int(splitline[inclnum])==0:
					continue
			tids.append(long(splitline[tidnum])-1)
			if importmultiloc:
				centers.append(int(splitline[centernum]))
				incls.append(int(splitline[inclnum]))
	tids = tids[startat:]
	if importmultiloc:
		centers = centers[startat:]
		incls = incls[startat:]
	maxpages = len(tids)
	tids.append(0)
	firsthit = tids[0]
	rf.close()
else:
	firsthit = float("+inf")

outcomes = {}
locnum = -1
if newmultiloc:
	tweetlist = [0]*len(loclist)
	searchesleft = [0]*len(loclist)
	mintids = [0]*len(loclist)
	maxtids = [0]*len(loclist)
elif importmultiloc:
	tweetlist = [0]*maxpages		#tweetlist[tweetnum][locnum][resnum] = [outline,loc,tid] for the resnum-th baseline tweet in loc locnum on testtweet tweetnum
	mintids = [0]*maxpages
	maxtids = [0]*maxpages
	

In [9]:
#Actual search-performing code
outfile = ''

if importcsv:
	if not outfile:
		outfile = 'base.'+term1.strip('\"')+'.'+importcsv
else:
	#if scheduled:
	#	scheddir = 'scheduled'
	#	if not os.path.exists(scheddir):
	#		os.mkdir(scheddir)
	#	outfile = scheddir+'/'+term1.strip('\"')+'.'+strftime('%Y%m%d.%H%M')+'.csv'
	if not outfile:
		outfile = term1.strip('\"')+'.csv'
wf = open(outfile,overwrite)
latlong = loclist[0]
#TODO: block out the header stuff
if header:
	wf.write('#Compiled by SeeTweet '+versionnum+'.\n')
	wf.write('#Search performed at '+strftime('%Y-%m-%d %H:%M')+'\n')
	if not multiloc:
		wf.write('#Search location: '+latlong+','+radius+'\n')
	else:
		wf.write('#Search location: U.S. 4-location coverage points (Texas, KC, SD, SF)\n')
	wf.write('#Search term: '+term1+'\n')
if (overwrite=='w' and not multiloc):
	wf.write('day,year,month,date,hour,minute,second,source,city,state,lat,long,uid,tid\n')
elif (overwrite=='w' and newmultiloc):
	wf.write('day,year,month,date,hour,minute,second,source,city,state,lat,long,uid,tid,center,incl\n')
elif (overwrite=='w' and importmultiloc):
	wf.write('day,year,month,date,hour,minute,second,source,city,state,lat,long,uid,tid,origtid,origincl,center,incl\n')
if keeptweets:
	tweetdir = 'tweetarchive'
	if not os.path.exists(tweetdir):
		os.mkdir(tweetdir)
	#if not os.path.exists(tweetdir+'/'+scheddir):
	#	os.mkdir(tweetdir+'/'+scheddir)
	outtweetfile = tweetdir+'/'+os.path.splitext(outfile)[0]+'.tweets'
	wft = open(outtweetfile,overwrite)
	if overwrite=='w':
		if raw:
			wft.write('day\tyear\tmonth\tdate\thour\tminute\tsecond\tloc\tuid\ttid\ttweet\ttlength\trt\n')
		else:
			wft.write('loc,uid,tid,tweet\n')
if trackfails:
	faildir = 'failures'
	if not os.path.exists(faildir):
		os.mkdir(faildir)
	outfailfile = faildir+'/'+os.path.splitext(outfile)[0]+'.fails'
	wff = open(outfailfile,'w')
	
	
	#Note: block out the rate check
for latlong in loclist:
	locnum = locnum + 1
	geocodestr = latlong+","+radius
	print "\nSearch centered at:", latlong, "(locnum "+str(locnum)+")"
	if newmultiloc:
		currloctweets = []
		tidbycurrloc = []
	
	for pagenum in range(0,maxpages):
		print ''
		if importmultiloc:
			if (locnum==0):
				tweetlist[pagenum] = []
				maxtids[pagenum] = []
				mintids[pagenum] = []
			#print maxtids
			currloctweets = []
			tidbycurrloc = []
		#Examining the rate limit
		if (pagenum % checklimits == 0):
			r = getlimits(tsearch)
			if r['remaining'] <= checklimits:
				print "\n\n**Paused because of rate limit.**"
				print "Current time:",strftime('%I:%M:%S')
				print "Reset time:  ",strftime('%I:%M:%S',localtime(r['reset']))
				if not multiloc:
					print "Resume with flag -s="+str(startat+pagenum)
				else:
					print "Stopped on location "+str(locnum)+", tweet "+str(startat+pagenum)+"/"+str(maxpages)
				print "--"
				print tweetcount, 'tweets found. Centered at', geocodestr
				print outcomes
				if throttle:
					waittime = r['reset']-time()+30
					print "Waiting", round(waittime), "seconds before resuming."
					sleep(waittime)
				else:
					sys.exit()
			elif r['remaining'] < 11:
				print "\n\n**WARNING:", r['remaining'], "queries remaining.**"
				print "Current time:",strftime('%I:%M:%S')
				print "Reset time:  ",strftime('%I:%M:%S',localtime(r['reset']))
				print ""
				print "\n\n"
				sleep(10)
		
		#Adding a catch for various Twitter errors
		errors = 0
		while (errors < MAXERRORS):
			try:
				if (firsthit == float("+inf")):
					res = tsearch.search.tweets(q=term+'+-rt',geocode=geocodestr,count=str(tweetspersearch),result_type="recent")
					#pprint(res['statuses'][99])
				else:
					#print 'Test:', term+'+-rt',geocodestr,str(tweetspersearch),str(firsthit)
					res = tsearch.search.tweets(q=term+'+-rt',geocode=geocodestr,count=str(tweetspersearch),result_type="recent",max_id=str(firsthit))
					#pprint(res['statuses'][99])
				break
			except TwitterHTTPError as e:
				errors = errors + 1
				print "Twitter Error encountered. Retrying",MAXERRORS-errors,"more times."                                                       
				print "\n"+e.response_data
				sleep(5)
		if (errors==MAXERRORS):
			print "Repeated errors encountered, possibly due to rate limit."
			print "Will wait 15 minutes and try once more before quitting."
			sleep(900)
			try:
				if (firsthit == float("+inf")):
					res = tsearch.search.tweets(q=term+'+-rt',geocode=geocodestr,count=str(tweetspersearch),result_type="recent")
				else:
					res = tsearch.search.tweets(q=term+'+-rt',geocode=geocodestr,count=str(tweetspersearch),result_type="recent",max_id=str(firsthit))
			except:			
				raise Exception("Gave up because of repeated errors, sorry.")
		res = res['statuses']
			
		print len(res), 'hits on page', startat+pagenum+1, '(max_id='+str(firsthit)+')'
		print r['remaining']-1, 'queries remaining.'
		if (len(res)==0):
			print 'Out of tweets at this location.'
			#print 'Test:', term+'+-rt',geocodestr,str(tweetspersearch),str(firsthit)
			#sleep(1)
			#res = tsearch.search.tweets(q=term+'+-rt',geocode=geocodestr,count=str(tweetspersearch),result_type="recent",max_id=str(firsthit))
			#pprint(res['statuses'][99])
			break
		for i in range(0,len(res)):
			tweetcount = tweetcount + 1
			[outline,outcome,tid,tline] = extractinfo(res[i],wff,raw)
			if outline:
				if not multiloc:
					wf.write(outline)
				elif newmultiloc:
					currloctweets.append([outline[:-1]+','+str(locnum)+'\n',locnum,long(tid)])
					tidbycurrloc.append(float(tid))
					#wf.write(outline[:-1]+','+str(locnum)+'\n')
				elif importmultiloc:
					currloctweets.append([outline[:-1]+','+str(tids[pagenum]+1)+','+str(incls[pagenum])+','+str(locnum)+'\n',locnum,long(tid)])
					tidbycurrloc.append(float(tid))
			if keeptweets:
				#wft.write(tline.encode('ascii','ignore'))
				if not raw:
					wft.write(tline.encode('ascii','ignore'))
				if raw:
					wft.write(tline.encode('ascii','replace'))		#maybe create a separate .utweets file with the Unicode versions of tweets
			outcomes[outcome] = outcomes.get(outcome,0)+1
			if importcsv:
				firsthit = tids[pagenum+1]
			else:
				if firsthit > long(tid):			#if current tweet came before previous oldest, update oldest
					firsthit = long(tid)-1
		if importmultiloc:
			if len(tidbycurrloc) > 0:
				maxtids[pagenum].append(max(tidbycurrloc))
				mintids[pagenum].append(min(tidbycurrloc))
			else:
				maxtids[pagenum].append(0)
				mintids[pagenum].append(0)				
			tweetlist[pagenum].append(currloctweets)
	#endfor searches within a location
	if newmultiloc:
		if len(tidbycurrloc) > 0:
			maxtids[locnum] = max(tidbycurrloc)
			mintids[locnum] = min(tidbycurrloc)
		else:
			maxtids[locnum] = 0
			mintids[locnum] = 0
		searchesleft[locnum] = maxpages-pagenum-1			#calculating how many pages were left in the most maxed-out search
		tweetlist[locnum] = currloctweets
	overwrite = 'a+'
	header = False
	if importcsv:
		firsthit = tids[0]
	else:
		firsthit = float("+inf")

#Endfor multiple locations
if newmultiloc:
	balanceandprint(tweetlist,mintids,maxtids,searchesleft,wf)
elif importmultiloc:
	for pagenum in range(0,maxpages):
		balanceandprint(tweetlist[pagenum],mintids[pagenum],maxtids[pagenum],[0]*len(loclist),wf)
wf.close()
if keeptweets:
	wft.close()
if trackfails:
	wff.close()
	


Search centered at: 30.8,-98.6 (locnum 0)

75 hits on page 1 (max_id=inf)
179 queries remaining.
CS: new york,ny 19 Breezy Outfits For When the Weather Is Just Too Hot to Handle https://t.co/op0MUB3MIB https://t.co/rXjpcvmUzL
CS: las vegas,nv It's hot outside, let me go swimming in your eyes
CS: shawnee,ok When it's hot outside and raining at the same time ..😩 https://t.co/9dUbLTT66v
F: toronto  ontario 736212357584654336
CS: las vegas,nv Watch Hard 4K Porn Videos For Free ➔ https://t.co/9lT4vOIYhU #porn #vip #movies #video #ass #cumtribute https://t.co/Nw5qPopxM7
F: minnesota 860550394006646784
CS: las vegas,nv Watch hard porn videos for free ➔ https://t.co/9lT4vOIYhU #hotgirl #hot #pussy #sext #videos #movies https://t.co/gcyWcApv0x
CS: phoenix,az Hot outside? Grab a refreshing... sewage popsicle? (Yuck; it's not really edible!) https://t.co/uShSeoBhAi https://t.co/3tIPYiKtFP
CS: phoenix,az Hot outside? Grab a refreshing... sewage popsicle? (Yuck; it's not really edible!) https://t.

CS: mountain home,ar It may be getting hot outside, but the HOTTEST deals are at Ultimate Chrysler Dodge Jeep RAM!!! https://t.co/u6FyKYdLZX
F: new york 16561306
CS: las vegas,nv adult vids in HD get your fREE access ➔ https://t.co/9lT4vOIYhU #nsfw #video #pussy #ass #movies #sexy #swallowcum https://t.co/37vs6zxbK9
F: colorado  usa 1066526090
F:  1145906497
CS: winter park,fl It's hot outside today. I'm slightly bored wanting a cold shower
CS: fontana,ca I feel like it's so hot outside & now I'm debating if I wanna dress like a thot today or put on actual clothes
F: canada 2601553358
F: utah  usa 447291916
F: jamaica 4214813592
F: midwest 362034298
CS: orlando,fl It's too damn hot outside
F: texas  usa 713197429
CS: scottsdale,az It's hot outside 👉 shade your head in style! https://t.co/Ycvp7gkE5x
CS: amarillo,tx It's way hot outside , I'm melting .
CS: austin,tx @nahitscooll It's hot outside!
F: overbrook  philadelphia 825706433278799872
LL: 27.220232,-80.795969 Ok it’s hot outside, 

CS: kansas city,mo It's hot outside an it looks like she's natural so it's possible that she could have sweated out her leave out 😑😑😑😑
CS: killeen,tx It may be getting hot outside, but don’t expect Killeen’s Pershing Park Pool to open up anytime soon. https://t.co/YUE3MoI7CP
F:  587198022
F: goderich ontario canada 172002568
KC: houston I'm deciding if I should go to pride this year or not it's gonna be so damn hot outside and I got no friends but idc
F: chicagoland usa 294907233
F: canada 3061926986
CS: mission,tx George always likes to go bike riding when it's really hot outside! I look darker😭
CS: chicago,il This that awkward weather where it's hot outside and freezing in every building
CS: kenedy,tx Dead ass sitting in the cooler at work bc it's so fkn hot outside 😭
CS: vinton,la it's so freaking hot outside
CS: grand rapids,mi 21 Cats That Are 100% You When It's Hot Outside https://t.co/5fRzJhl9kX
F: ottawa  canada 19487948
F: texas  usa 1678568611
CS: new orleans,la Why it gotta 

F: toronto  ontario 848340199826493441
CS: houston,tx So damn hot outside 😅
CS: san antonio,tx fuckin hot outside
CS: lexington,ky It's hot outside, but the sky is blue. I'd call that a double standard!
CS: minneapolis,mn @Dish_AWD BUT ITS HOT OUTSIDE!!
F: staten island 529078896
F: north carolina 4868361670
CS: wheaton,md It may be hot outside, but you can stay cool at the @WheatonIce  arena! https://t.co/EXBK6v88J6
CS: dallas,tx ⚠️it's getting quite hot outside so please do not leave anything that breathes in a hot car, even if it's just for a few minutes!!⚠️
F: avenue de la tristesse 295461578
CS: new orleans,la It's #sunny and #hot outside! It's the perfect day for a #snoball ! Find your favorite snoball flavor of... https://t.co/6GOLBY4LPG
F: toronto  ontario 79853685
F: 🗣dat985! 3048481119
F: bloomington  minnesota  usa 21969606
F: hill valley 353339455
KC: detroit @iamthedriving What is there to get? It's a park with slides and play sculptures that also has water, which is appe…

75 hits on page 8 (max_id=872578008699342851)
173 queries remaining.
KC: el paso It's so hot outside, let me float naked in your pool while drinking a 40oz.
CS: houston,tx @Jzacharyyrahcaz You right, us Mexicans were BORN to work lol.. but still to hot outside for me 🤷🏻‍♂️
CS: tampa,fl How to Manage Your Diabetes When It's Hot Outside https://t.co/NQkqNYqW6v https://t.co/UBOdstOQt3
F: toronto 162395712
CS: goldsby,ok It's hot outside, but we've got cold drinks and good music!! Open mic starts at 7 with Dwight Hamlin!! https://t.co/pDmhSBHogP
CS: tampa,fl How to Manage Your Diabetes When It's Hot Outside https://t.co/NQkqNYqW6v https://t.co/bdKy4ZUVnj
CS: spokane valley,wa It may be getting hot outside, but you can still have cool styles for less! Shop our 50% off Clearance sale NOW! https://t.co/SLeNXZ9J03
F:  57213222
CS: las vegas,nv Just a lil stinkin hot outside but McDonald's Fries sounded soo good! Fat, Full & happy🐷🐷🐷
F: kingston  jamaica 53140952
F: texas 2728610300
CS: tampa,f

...

CS: houston,tx My windows need to be tinted it's too damn hot outside
F: south texas 820435111
F: united states 863262061341663232
KC: omaha @CitizenRyan82 i know it’s not that hot outside, but man, that radiant heat off the parking lot and the car sitting… https://t.co/13TPzbofHL
CS: brownsville,tx It is way too hot outside
CS: riverside,ca Can someone tell me if it's hot outside I'm too lazy to get up
CS: alexandria,mn Couch to 5k: Baby, It's HOT Outside -  https://t.co/ixyWmVkOHt
KC: las vegas It's so hot outside!! 🌞
F: united states 4723499898
CS: la feria,tx It's so hot outside😩
F: texas  usa 1717128186
CS: corpus christi,tx Bright day out side, stay cool little birds. It's hot outside 💋 https://t.co/ymmF4XfU7N
KC: las vegas When it's this hot outside cool off with a cold Drink! https://t.co/NELrgo7yLv
CS: mesa,az It's Monday and it's hot outside. Come let our drink specials & misted patio keep you cool! 1/2 price liquor & $2.5… https://t.co/IXMOZi1mr8
CS: pahrump,nv Whoops, it's 

F: wit yo bitch 736415577816731648
F: o'connor 719723483287187457
F: united states 871206583836999682
CS: corpus christi,tx ITS SO HOT OUTSIDE. It's perfect for swimming. 🙆‍♂️
CS: reno,nv Good thing it's hot outside. I spilled a gallon of water on the front of my pants right before I started lunch. #fml
F: united states 2582911394
F: the castle 325203607
CS: austin,tx Fuck it.... it's too hot outside... this how I'm comin today https://t.co/4fwkIlXS34
CS: houston,tx @iniguezmari_ Good mama because it's hot outside and he needs air condition 😤
F: if only i was cool & in gc's 4687965025
F: usa 731707082492252161
CS: boise,id You know it's hot outside when you got outside & it's hot outside
CS: new orleans,la It's fucking hot outside
CS: minneapolis,mn @MeatSauce1, it's too hot outside. BTsquaredF2 is on TBS. #powertrip
KC: st louis Hot outside
F: united states 106570187
CS: houston,tx It's stupid hot outside ..
F: canada 375442181
CS: liberal,ks @Gods___gift @GanjaGoddessbae @caamaachhoo

F: jalisco 703588026
GEO: [49.97607127, -125.23596953] Hot outside #nostresshere @ Willow Point Sportsplex https://t.co/kjWtn1jCef
CS: san antonio,tx It is TOO hot outside I don't think I'm ready for summer, help
CS: houston,tx Like damn man my whole attitude is not going and it's fucking  hot outside
KC: st louis mo It's too damn hot outside
CS: fairfield,ia It's hot outside, come join us on the patio for some cold drinks, we have mojitos , adult lemonade and Moscow mules. https://t.co/Uz1KT7Gr3j
CS: bloomington,ca It's too fuckin hot outside!
CS: san antonio,tx Too Hot Outside? Book a Penguin Interaction today to cool off and meet some new friends! Pass members save 20%! https://t.co/QJAC11VGO8
CS: tucson,az It's 9:30 & it's already hot outside 😩 #AZProbz
F: astoria 30923399
KC: st louis mo Playing that hot outside, freezing cold inside game--yay for June! Today I layered a basic… https://t.co/SBhQli6WtA
CS: houston,tx It's so fucking hot outside ITS RIDICULOUS
CS: las vegas,nv it's 

F: vancouver  b.c. 16152163
CS: omaha,ne I know it's June and it's starting to get hot outside, but it's not summer until I hear the ice cream truck driving down the street.
CS: visalia,ca It's too hot outside 😭
CS: hopkins,mn @TheCrystai it's so hot outside in the upper 80's☀️😎staying inside where it cool at gym workout u are getting me motivated 2💪🏼🥊workout again
F: us of a 568777982
F: united states 717773405425115136
F: alberta  canada 1650761390
CS: waco,tx Soon, it'll be super hot outside and providing fresh, clean water is very important! We proudly sell JUG... https://t.co/UvM6b9pGXy
CS: mission viejo,ca why is it so mf hot outside 🙁
KC: st louis When it's too hot outside to go for a walk but you decided to wear your fur coat anyway https://t.co/9Ozw3qL0EF
CS: avondale,az Bruh its hot outside already
F: humboldt  sk 337282307
CS: portland,or @evanempdx "People are saying, 'it's hot outside', and I'm just like, 'that's the sun'... idk what everyone's in a fuss about."
CS: des mo

9 hits on page 17 (max_id=870217121212641279)
106 queries remaining.
F: devon 423625037
GEO: [34.697781, -118.143675] ☀️Welcome June & Summer!☀️ It's gonna get HOT outside!🔥 Keep your skin protected, get your… https://t.co/uCz1aPk3Oi
CS: las vegas,nv Watch Hard Porn Videos For Free ➔ https://t.co/9lT4vP0z9s #amateur #videos #tits #porn https://t.co/zw4P4qmCuA
CS: dallas,tx its HOT outside, time to shave head
CS: lafayette,la Why tf is it so hot outside
F: tennessee  usa 704315318118830080
F: united states 823476844452118529
F: canada 54085062
KC: dallas it's too fucking hot outside   i can't do it

0 hits on page 18 (max_id=869976105851080703)
106 queries remaining.
Out of tweets at this location.
Balancing multi-location search results.
maximin: 2.28661410997e+15
16 cutoffs for loc 0
0 cutoffs for loc 1
350 cutoffs for loc 2
336 cutoffs for loc 3


### Now go to R to finish processing

In [7]:
scheduled

False

In [29]:
print outfile

good+morning.csv
